In [1]:
#Importing all the dependancies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from scipy import stats
import statsmodels.api as sm
import math
import re
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor
import sys
import pandas.core.algorithms as algos
pd.pandas.set_option('display.max_columns',None)

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Vishal
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#Loading Dataset
#df = pd.read_csv('/content/drive/MyDrive/train.csv')
#test_data = pd.read_csv('E:\Fake News Detection/test.csv')
#submit = pd.read_csv('E:\Fake News Detection/submit.csv')
#df = pd.read_csv('train.csv')
df = pd.read_csv('E:\Fake News Detection/train.csv')


In [4]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


# Data Preprocessing

In [5]:
df = df.fillna('')

In [6]:
#merging the train and test data across rows
#df = pd.concat([train_data, test_data], axis = 'rows')
#df.info()

In [7]:
#merging the "author" and "title" columns to form "Content"
df['content'] = df['author']+' '+df['title'] #+df['text']
df['content'].head()

0    Darrell Lucus House Dem Aide: We Didn’t Even S...
1    Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2    Consortiumnews.com Why the Truth Might Get You...
3    Jessica Purkiss 15 Civilians Killed In Single ...
4    Howard Portnoy Iranian woman jailed for fictio...
Name: content, dtype: object

In [8]:
# separating the data & label
X = df.drop(columns='label', axis=1)
y = df['label']

In [9]:
#Defining a function "clean" to stem to the words in content
from nltk.stem import WordNetLemmatizer

stemmer = PorterStemmer()
wordnet = WordNetLemmatizer
def clean(text):
    text="".join([re.sub('[^a-zA-Z]',' ',char) for char in text])
    text=text.lower()
    text=text.split()
    text=[stemmer.stem(word) for word in text if word not in set(stopwords.words("english"))]
    text=" ".join(text)
    return text

In [10]:
df['content'] = df['content'].apply(clean)

In [11]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [12]:
#Separating lables and values
X = df['content'].values
y = df['label'].values

In [13]:
print(X)
print(y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [14]:
y.shape

(20800,)

In [15]:
#Converting the textual data into numerical data using vertorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [16]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [17]:
#Splitting test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify=y, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, stratify=y_train, random_state=42)

# Implementing Logistic Regression

In [74]:
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [75]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, y_train)
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9874399038461539


In [76]:
#Accuracy score on test data
X_test_pred = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_pred, y_test)
print ('Accuracy score of the testing data :', test_data_accuracy)

Accuracy score of the testing data : 0.9752403846153846


In [78]:
#Implementing Bagging ensemble on Logistic Regression Classifier
from sklearn.ensemble import BaggingClassifier
bagging_clf = BaggingClassifier(base_estimator = LogisticRegression(), n_estimators = 100, max_features = 15000, random_state = 42)
bagging_clf.fit(X_train, y_train).score(X_test, y_test)

0.9711538461538461

# Implementing XGBoost

In [18]:
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [19]:
XGBClassifier().get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': False,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [21]:
model_xgboost = xgboost.XGBClassifier(n_estimators = 100,
                                     eval_metric = 'auc')
#eval_set =[(X_val, y_val)]
model_xgboost.fit(X_train,
                  y_train,
                  #early_stopping_rounds=10,
                  #eval_set=eval_set,
                  verbose=True)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [22]:
y_train_pred=model_xgboost.predict_proba(X_train)[:,1]
y_test_pred=model_xgboost.predict_proba(X_test)[:,1]
print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

y_train_pred=model_xgboost.predict(X_train)
y_test_pred=model_xgboost.predict(X_test)
print("Accuracy Train: {:.4f}\nAccuracy Test: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))

AUC Train: 0.9994
AUC Test: 0.9950
Accuracy Train: 0.9947
Accuracy Test: 0.9875


## Cross validation result without hyperparameter tuning

In [23]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(model_xgboost, X, y, cv =10)

In [24]:
score.mean()

0.9891826923076923

In [86]:
#getting the parameters for the defaul XGBoost model
XGBClassifier().get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': False,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

## Hyperparameter tuning on XGBoost

In [87]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold


In [88]:
classifier = XGBClassifier()

In [111]:
#Selecting superset of parameters to train on
params = {
    "learning_rate" : [0.01, 0.10, 0.25, 0.5, 1],
    "max_depth" : [3,5,10],
    "min_child_weight" : [1,3,5,7, 10, 13,17, 20],
    "gamma" : [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
    "colsample_bytree" : [0.3, 0.4, 0.5, 0.7, 1],
    
}

### RandomizedSearchCV

In [112]:
#Hyperparameter tuning using Randomized Search CV
random_search =  RandomizedSearchCV(classifier, 
                                    param_distributions = params, 
                                    n_iter=10, 
                                    scoring ="accuracy", n_jobs =-1, cv=10, verbose =3)

In [113]:
random_search.fit(X,y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   46.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.9min finished


RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           callbacks=None, colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=0.7,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric='auc', gamma=0.2,
                                           gpu_id=-1, grow_policy='depthwise',
                                           importance_type=None,
                                           interaction_constraints='',
                                           learning_rate=1,...
                   iid='deprecated', n_iter=10, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.3, 0.4, 0.5, 0.7,
                                         

In [114]:
#Getting the best params from Random search
random_search.best_params_

{'min_child_weight': 1,
 'max_depth': 5,
 'learning_rate': 0.5,
 'gamma': 0.2,
 'colsample_bytree': 1}

In [115]:
#Fitting the model with the best params we got
classifier = xgboost.XGBClassifier(min_child_weight= 1,
  max_depth= 5,
  learning_rate= 0.5,
  gamma= 0.2,
  colsample_bytree= 1,n_estimators = 100,eval_metric = 'auc')

In [116]:
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0.2, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.5, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [117]:
y_train_pred=classifier.predict(X_train)
y_test_pred=classifier.predict(X_test)
print("Training accuracy after tuning the parameters using GridSearchCV: {:.4f}\nTest accuracy after tuning the parameters using GridSearchCV: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))
                                            

Training accuracy after tuning the parameters using GridSearchCV: 0.9957
Test accuracy after tuning the parameters using GridSearchCV: 0.9873


In [118]:
score_r = cross_val_score(classifier, X, y, cv =10, scoring = 'accuracy')

In [119]:
# Why is this higher? 
score_r.mean() #0.9872115384615385

0.9904807692307692

In [ ]:
# Only to be used on validation set
#eval_set =[(X_val, y_val)]
#classifier.fit(X_train,
                  #y_train,
                  #early_stopping_rounds=10,
                  #eval_set=eval_set,
                  #verbose=True)

In [ ]:
# If predicting using this then train the model using only train data. 
# When doing cross validation, it is trained on X, rather than X_train. 
# Cause, CV handles the split by itself. 
#y_train_pred=classifier.predict_proba(X_train)[:,1]
#y_test_pred=classifier.predict_proba(X_test)[:,1]
#print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))

#y_train_pred=classifier.predict(X_train)
#y_test_pred=classifier.predict(X_test)
#print("Accuracy Train: {:.4f}\nAccuracy Test: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))

### GridSearchCV

In [120]:
classifier.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': False,
 'base_score': 0.5,
 'booster': 'gbtree',
 'callbacks': None,
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': 'auc',
 'gamma': 0.2,
 'gpu_id': -1,
 'grow_policy': 'depthwise',
 'importance_type': None,
 'interaction_constraints': '',
 'learning_rate': 0.5,
 'max_bin': 256,
 'max_cat_to_onehot': 4,
 'max_delta_step': 0,
 'max_depth': 5,
 'max_leaves': 0,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 0,
 'num_parallel_tree': 1,
 'predictor': 'auto',
 'random_state': 0,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'sampling_method': 'uniform',
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None}

In [ ]:
random_search.best_params_

In [126]:
params2 = {
    "learning_rate" : [0.25, 0.5, 1],
    "max_depth" : [3,5,9],
    "min_child_weight" : [1,3],
    "gamma" : [0.1, 0.2, 0.5],
    "colsample_bytree" : [ 0.5, 1]
    
}

In [127]:
grid_search = GridSearchCV( 
    estimator = classifier, param_grid = params2, scoring ="accuracy", n_jobs =-1, cv=5, verbose =3)

grid_search = grid_search.fit(X_train,y_train)

#print(model.best_score_)
#print(model.best_estimator_.get_params())

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:   27.8s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:  2.1min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done 504 tasks      | elapsed: 12.0min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 13.0min finished


In [128]:
grid_search.best_params_


{'colsample_bytree': 1,
 'gamma': 0.5,
 'learning_rate': 0.5,
 'max_depth': 9,
 'min_child_weight': 1}

best parameters so far -- {'colsample_bytree': 0.5,
 'gamma': 0.5,
 'learning_rate': 0.5,
 'max_depth': 11,
 'min_child_weight': 1}

In [129]:
#Fitting the model with the best params we got
classifier = xgboost.XGBClassifier(min_child_weight= 1,
  max_depth= 9,
  learning_rate= 0.5,
  gamma= 0.5,
  colsample_bytree= 1,n_estimators = 100,eval_metric = 'auc')

In [130]:
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0.5, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.5, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=9, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [131]:
y_train_pred=classifier.predict(X_train)
y_test_pred=classifier.predict(X_test)
print("Training accuracy after tuning the parameters using GridSearchCV: {:.4f}\nTest accuracy after tuning the parameters using GridSearchCV: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))
                                            

Training accuracy after tuning the parameters using GridSearchCV: 0.9978
Test accuracy after tuning the parameters using GridSearchCV: 0.9870


In [138]:
#Using Stratified Kfold validation for accuracy
skf = StratifiedKFold(n_splits=10)

score_g = cross_val_score(classifier, X, y, cv =skf, scoring = 'accuracy')

In [139]:
score_g.mean() #0.9915865384615385

#score_g.max()

0.9910096153846153

In [142]:
#eval_set =[(X_val, y_val)]
#classifier.fit(X_train,
                  #y_train,
                  #early_stopping_rounds=10,
                  #eval_set=eval_set,
                  #verbose=True)

In [ ]:
#Accuracy with GridSearchCV hyperparameter tuning
#y_train_pred=classifier.predict_proba(X_train)[:,1]
#y_test_pred=classifier.predict_proba(X_test)[:,1]
#print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),
                                                    roc_auc_score(y_test, y_test_pred)))


#y_train_pred=classifier.predict(X_train)
#y_test_pred=classifier.predict(X_test)
#print("Training accuracy after tuning the parameters using GridSearchCV: {:.4f}\nTest accuracy after tuning the parameters using GridSearchCV: {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))
                                            

### Bayesian Optimization using HyperOpt

In [144]:
from sklearn.metrics import precision_recall_fscore_support as score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, space_eval

In [155]:
# Space
space = {
    'learning_rate': hp.choice('learning_rate', [ 0.01, 0.1, 0.25, 0.5, 1]),
    'max_depth' : hp.choice('max_depth', [3,5,9,11]),
    'min_child_weight' : hp.choice('min_child_weight', range(1,4)),
    'gamma' : hp.choice('gamma', [i/10.0 for i in range(0,8)]),
    'colsample_bytree' : hp.choice('colsample_bytree', [i/10.0 for i in range(3,11)])
    #'reg_alpha' : hp.choice('reg_alpha', [0, 1e-5, 1e-2, 0.1, 1, 10, 100]), 
    #'reg_lambda' : hp.choice('reg_lambda', [1e-5, 1e-2, 0.1, 1, 10, 100])
}
# Set up the k-fold cross-validation
kfold = StratifiedKFold(n_splits=5)
# Objective function
def objective(params):
    
    xgboost = XGBClassifier(seed=42, **params)
    scores = cross_val_score(xgboost, X_train, y_train, cv=kfold, scoring='accuracy', n_jobs=-1)
    # Extract the best score
    best_score = max(scores)
    # Loss must be minimized
    loss = - best_score
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}
# Trials to track progress
bayes_trials = Trials()
# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, max_evals = 100, trials = bayes_trials)

100%|█████████████████████████████████████████████| 100/100 [16:50<00:00, 10.10s/trial, best loss: -0.9936899038461539]


In [156]:
# Print the index of the best parameters
# print(best)
# Print the values of the best parameters
print(space_eval(space, best))


{'colsample_bytree': 0.5, 'gamma': 0.0, 'learning_rate': 0.5, 'max_depth': 11, 'min_child_weight': 1}


In [157]:
#Fitting the model with the best params we got
classifier = xgboost.XGBClassifier(min_child_weight= 1,
  max_depth= 11,
  learning_rate= 0.5,
  gamma= 0.,
  colsample_bytree= 0.5,
  n_estimators = 100,eval_metric = 'auc')

In [158]:
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='auc', gamma=0.0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.5, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=11, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [159]:
y_train_pred=classifier.predict(X_train)
y_test_pred=classifier.predict(X_test)
print("Training accuracy after tuning the parameters using Bayesian Optimizer : {:.4f}\nTest accuracy after tuning the parameters using Bayesian Optimizer : {:.4f}".format(accuracy_score(y_train, y_train_pred),
                                                    accuracy_score(y_test, y_test_pred)))

Training accuracy after tuning the parameters using Bayesian Optimizer : 0.9984
Test accuracy after tuning the parameters using Bayesian Optimizer : 0.9880


In [164]:
#Using Stratified Kfold validation for accuracy
skf = StratifiedKFold(n_splits=10)
score_b = cross_val_score(classifier, X, y, cv =skf, scoring = 'accuracy')

In [165]:
score_b.mean()
#score_b.max()

0.9915865384615385

In [166]:
#eval_set =[(X_val, y_val)]
#classifier.fit(X_train,
                  #y_train,
                  #early_stopping_rounds=10,
                  #eval_set=eval_set,
                  #verbose=True)

In [169]:
#y_train_pred=classifier.predict_proba(X_train)[:,1]
#y_test_pred=classifier.predict_proba(X_test)[:,1]
#print("AUC Train: {:.4f}\nAUC Test: {:.4f}".format(roc_auc_score(y_train, y_train_pred),roc_auc_score(y_test, y_test_pred)))
                                                    


#y_train_pred=classifier.predict(X_train)
#y_test_pred=classifier.predict(X_test)
#print("Training accuracy after tuning the parameters using Bayesian Optimizer : {:.4f}\nTest accuracy after tuning the parameters using Bayesian Optimizer : {:.4f}".format(accuracy_score(y_train, y_train_pred), accuracy_score(y_test, y_test_pred)))
                                                    

## TPOT Classifier

In [25]:
import torch
from torch import nn

from tpot import TPOTClassifier

In [27]:
params_tpot = {
    "learning_rate" : [0.1, 0.25, 0.5, 1],
    "max_depth" : [3,5,9,11],
    "min_child_weight" : [1,3],
    "gamma" : [0.0,0.1,0.3, 0.5, 0.6, 0.7],
    "colsample_bytree" : [0.4, 0.5, 0.7, 1]
    
}

In [ ]:
tpot_classifier = TPOTClassifier(generations =10, population_size = 24, offspring_size = 12,
                              verbosity = 2, early_stop = 12, random_state = 42,
                              config_dict = {'xgboost.XGBClassifier' : params_tpot},
                               cv = 5, scoring = 'accuracy') #cv = 5

#skf = StratifiedKFold(n_splits=5)
#score_tpot = cross_val_score(tpot_classifier, X, y, cv =skf, scoring = 'accuracy')
tpot_classifier.fit(X_train, y_train) ##if cv = 5 is passed as a parameter inside TPOTClassifier then run this. 

In [26]:
accuracy = tpot_classifier.score(X_test, y_test)
print(accuracy)
#print(score_tpot.mean())

NameError: name 'tpot_classifier' is not defined

In [ ]:
skf = StratifiedKFold(n_splits=10)
score_tpot = cross_val_score(tpot_classifier, X, y, cv =skf, scoring = 'accuracy')


Generation 1 - Current best internal CV score: 0.9887620192307693

Generation 2 - Current best internal CV score: 0.9887620192307693

Generation 3 - Current best internal CV score: 0.9887620192307693

Generation 4 - Current best internal CV score: 0.9888822115384615

Generation 5 - Current best internal CV score: 0.9888822115384615

Best pipeline: XGBClassifier(input_matrix, colsample_bytree=0.4, gamma=0.5, learning_rate=0.5, max_depth=11, min_child_weight=1)


In [ ]:
score_tpot.mean()

## Optuna
